# NeuroMet

## Example directory tree

### External Modules

In [3]:
import sys, os, glob 
sys.path.append(os.path.abspath('./'))
#from neuromet import scanner_to_bids
from neuromet import NeuroMet2_dev_fs7

In [ ]:
! which recon-all

In [ ]:
from IPython.display import Image
%matplotlib inline

### Settings

In [ ]:
## Paths
base_dir = os.path.join(os.getcwd(), 'NeuroMET')
temp_dir = os.path.join(os.getcwd(), 'tmp')

#cores
cores = 3
omp_nthreads = 3

#
subject_list = ['003T1']



# SPM Segment and Mask generation

### Tissue Segmentation Pipeline

In [ ]:
neuromet = NeuroMet2_dev_fs7.NeuroMet(sublist = subject_list,
                            temp_dir = temp_dir,
                            bids_root = base_dir,
                            omp_nthreads=3)

In [ ]:
neuromet = neuromet.make_neuromet1_workflow()

In [ ]:
neuromet.write_graph(graph2use = 'colored', dotfilename = './imgs/neuromet')
Image(filename='./imgs/neuromet.png', width=250)

In [ ]:
%%time
neuromet.run('MultiProc', plugin_args={'n_procs': cores})

# Combined image and Freesurfer analysis

In [ ]:
print(subject_list)

In [ ]:
#subject_list += ['012T3']
subject_list

In [ ]:
import importlib
importlib.reload(NeuroMet2_dev_fs7)

In [ ]:
neuromet = NeuroMet2_dev_fs7.NeuroMet(sublist = subject_list,
                            temp_dir = temp_dir,
                            bids_root = base_dir,
                            omp_nthreads=omp_nthreads)
neuromet_fs = neuromet.make_neuromet_fs_workflow()

In [ ]:
neuromet_fs.write_graph(graph2use = 'colored', dotfilename = './imgs/neuromet_fs')
Image(filename='./imgs/neuromet_fs.png', width=350)

**__Before running this step the file derivates/NeuroMET/masks.tsv should be updated with the best mask__**

In [ ]:
%%time
neuromet_fs.run('MultiProc', plugin_args={'n_procs': cores})

# Extract Volumes

In [ ]:
os.getcwd()

In [ ]:
from neuromet.nodes import stats
import pandas as pd

In [ ]:
import importlib
importlib.reload(stats)

In [ ]:
q = stats.Stats()

In [ ]:
q.inputs.basedir = os.path.join(base_dir, 'derivatives', 'NeuroMET_FS')
q.inputs.fs_dir_template = '/*/*.reconall'

In [ ]:
os.path.join(q.inputs.basedir, '*/*')

In [ ]:
glob.glob(os.path.join(q.inputs.basedir, '*/*.reconall'))

In [ ]:
res = q.run()

In [ ]:
res.outputs.stats_directory

In [ ]:
res.outputs

In [ ]:
dats = [i for i in os.listdir(res.outputs.stats_directory) if i.endswith('.dat')]

In [ ]:
dats

In [ ]:
for i in dats:
    i = os.path.join(res.outputs.stats_directory, i)
    pd.read_csv(i, sep='\t').to_excel(i.replace('.dat', '.xlsx'))

In [ ]:
os.path.isdir(res.outputs.stats_directory)